In [6]:
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
import sys
sys.path.append('/n/data1/hms/dbmi/zitnik/lab/users/vau974/NovelDDI/')
from novelddi.models import models
from torch import nn
import copy

import pandas as pd
import numpy as np

00:15:19   Note: NumExpr detected 28 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [9]:
kg = pd.read_csv('/n/data1/hms/dbmi/zitnik/lab/users/yeh803/DDI/PrimeKG/kg.csv')
meta = pd.read_csv('/n/data1/hms/dbmi/zitnik/lab/users/yeh803/DDI/processed_data/views_features/combined_metadata_reindexed_ddi.csv')

/home/vau974/.conda/envs/novelddi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (1,2,3,4,5,6,7,10,11,12,13,14,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
in_kg = meta[meta.view_kg == 1].node_id

In [16]:
kg = kg[kg.x_id.isin(in_kg)]

In [22]:
counts = kg.groupby('x_id')['x_id'].count().tolist()

In [27]:
np.save('counts.npy', np.array(counts))

In [26]:
import matplotlib.pyplot as plt
plt.hist(counts)
plt.show()

/tmp/ipykernel_2030/3999057193.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [19]:
VIEWS_PATH = '/n/data1/hms/dbmi/zitnik/lab/users/yeh803/DDI/processed_data/views_features/KG_data_hgt.pt'

In [20]:
graph = torch.load(VIEWS_PATH)
graph_copy = copy.deepcopy(graph)

In [21]:
# rev_added = []
# for edge in graph.edge_types:
#     if edge[0] == edge[2]:
#         x = graph[edge[0], edge[1], edge[2]].edge_index
#         x = torch.roll(x, 1, 0)
#         rev_added.append((edge[0], 'rev_' + edge[1], edge[2]))
#         graph[edge[0], 'rev_' + edge[1], edge[2]].edge_index = x
#         #graph[edge[0], 'rev_' + edge[1], edge[2]].edge_index = None

In [22]:
edge_types = []
rev_edge_types = []
for edge in graph.edge_types:
    if 'rev' not in edge[1]:
        edge_types.append(edge)
        rev_edge_types.append(None)
        #rev_edge_types.append( (edge[2], 'rev_' + edge[1] ,edge[0])  )

In [23]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected = False,
    disjoint_train_ratio=0.1,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=edge_types,
    rev_edge_types=rev_edge_types, 
)
train_data, val_data, test_data = transform(graph)

In [56]:
# def remove_rev(data):
#     for edge in data.edge_types:
#         if edge in rev_added:
#             del data[edge]
        
# remove_rev(train_data)
# remove_rev(val_data)
# remove_rev(test_data)

In [24]:
def manually_add_rev_labels(data):
    for edge in data.edge_types:
        if 'rev' in edge[1]:
            normal_edge = edge[1].replace("rev_", "")
            normal = (edge[2], normal_edge, edge[0])
            normal_edge_index = data[normal].edge_label_index
            data[edge]['edge_label_index'] = torch.roll(normal_edge_index, 1, 0)
            data[edge]['edge_label'] = data[normal].edge_label

def manually_add_rev_labels_2(data):
    for edge in data.edge_types:
        if 'rev' in edge[1]:
            pos = data[edge]['edge_index']
            #neg = torch.stack([torch.arange(data[edge]['edge_index'].max()), 
                               #torch.randperm(data[edge]['edge_index'].max())], dim=0)
            #data[edge]['edge_label_index'] = torch.cat([pos, neg], axis=1)
            data[edge]['edge_label_index'] = pos
            data[edge]['edge_label'] = torch.ones(pos.shape[1])
            #data[edge]['edge_label'] = torch.cat([torch.ones(pos.shape[1]), torch.zeros(neg.shape[1])])
            
manually_add_rev_labels(train_data)
manually_add_rev_labels(val_data)
manually_add_rev_labels(test_data)

In [25]:
train_edge_label_index = [ (train_data[edge]['edge_label_index'],edge) for edge in train_data.edge_types]
val_edge_label_index = [ (val_data[edge]['edge_label_index'],edge) for edge in val_data.edge_types]
test_edge_label_index = [ (test_data[edge]['edge_label_index'],edge) for edge in test_data.edge_types]

In [26]:
def check_data(data):
    for edge in data.edge_types:
        left = data[edge[0]]['num_nodes']
        right = data[edge[2]]['num_nodes']
        
        edge_index = data[edge]['edge_index']
#         print(edge)
#         print(edge_index[0,:].max(), left)
#         print(edge_index[1,:].max(), right)
        if edge_index[0,:].max() > left: print('hi')
        if edge_index[1,:].max() > right: print('hi')

check_data(train_data)
check_data(val_data)
check_data(test_data)

In [27]:
class HGTLinkPred(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, num_heads, 
                 metadata, num_edge_types, group='sum'):
        super(HGTLinkPred, self).__init__()
        
        self.encoder = models.HGT(in_channels, hidden_channels, out_channels, 
                                  num_layers, num_heads, metadata, group)
        self.decoder = models.BilinearDDIScorer(out_channels, out_channels, 1)
        self.decoders = [self.decoder for i in range(num_edge_types)]
    
    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        preds = []
        for decoder, (edge_pred_index, edge_name) in zip(self.decoders, edge_label_index):
            i, r, j = edge_name
            print(edge_name)
            node_in, node_out = edge_pred_index
            pred = decoder(z_dict[i], z_dict[j]).squeeze(0)
            pred = pred[node_in, node_out]
            preds.append(pred)
        return torch.cat(preds, 0)
    
    def save_checkpoint(self, PATH):
        torch.save(self.encoder.state_dict(), PATH)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = HGTLinkPred(in_channels=train_data.x_dict['drug'].shape[1],
                    hidden_channels=128,
                    out_channels=128,
                    num_layers=2,
                    num_heads=4,
                    num_edge_types=len(train_edge_label_index),
                    metadata = train_data.metadata()).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

loss_fn = nn.MSELoss()

def train():
    model.train()
    optimizer.zero_grad()
    preds = model(train_data.x_dict, train_data.edge_index_dict, train_edge_label_index)
    targets = torch.stack([i[0] for i in train_edge_label_index], 0)
    loss = loss_fn(pred, target) 
    loss.backward()
    optimizer.step()
    return float(loss)

In [29]:
for epoch in range(1, 301):
    loss = train()

('anatomy', 'anatomy_anatomy', 'anatomy')
('gene/protein', 'anatomy_protein_absent', 'anatomy')
('anatomy', 'rev_anatomy_protein_absent', 'gene/protein')


KeyboardInterrupt: 